# 构建数据集

In [ ]:
# 设置python的工作路径
import os

os.chdir("/home/huabei/projects/SMTarRNA")

## 导入包

In [ ]:
import pickle
from functools import partial

import pandas as pd
from tqdm import tqdm

from utils import ZincPdbqt, ele_filter, generate_coor, read_dock_score, write_data

os.getcwd()

## 从pkl文件读入对接能量分数

In [ ]:
# origin_100K_data_path = 'data//100k'
# origin_100K_data_path = 'data/3a6p/100k'
origin_100K_data_path = "../data/4z4d/100k_apH"

dock_score_100k = read_dock_score(origin_100K_data_path)
# dock_score_10k = read_dock_score(origin_10K_data_path)
score_dict = dock_score_100k
# with open('E:/Python_Project/SMTarRNA/project/data/3a6p/3a6p_dock_origin_data/zinc_drug_like_3d_100k_dock_energy.pkl', 'rb') as f:
#     score_dict = pickle.load(f)

In [ ]:
# 存入csv
# dataframe = pd.Series(score_dict)
# dataframe.to_csv('drug_like_100k_energy.csv')

In [ ]:
energy_d = pd.read_csv("../data/drug_like_100k_energy.csv")
smiles_d = pd.read_csv("../data/zinc_600m_100k_smiles.csv")
smiles_d.set_index("zinc_id", inplace=True)
smiles_d["smiles_len"] = smiles_d["smiles"].apply(lambda x: len(x), 0)
smiles_d["smiles_len"] = smiles_d["smiles_len"].astype("int64")

In [ ]:
smiles_d_cleaned = pd.Series()
for zinc_id in tqdm(energy_d["zinc_id"]):
    # print(zinc_id)
    # print(smiles_d.loc[zinc_id]['smiles_len'].argmin())
    if type(smiles_d["smiles"].loc[zinc_id]) == str:
        smiles_d_cleaned[zinc_id] = smiles_d["smiles"].loc[zinc_id]
    else:
        smiles_d_cleaned[zinc_id] = (
            smiles_d["smiles"].loc[zinc_id].iloc[smiles_d.loc[zinc_id]["smiles_len"].argmin()]
        )

    # break

In [ ]:
# energy_d.columns
smiles_d_cleaned

In [ ]:
# smiles_d_cleaned['energy'] = energy_d['energy']
# energy_d.set_index('zinc_id', inplace=True)
# out_dataframe = pd.DataFrame(data=dict(smiles=smiles_d_cleaned, energy=energy_d['energy']))
# out_dataframe = pd.merge(left=smiles_d_cleaned, right=energy_d['energy'], )
out_dataframe = pd.concat([smiles_d_cleaned, energy_d["energy"]], axis=1)
# energy_d['smiles'] = smiles_d_cleaned
out_dataframe.to_csv("drug_like_100k_smiles_dataset.txt", index=False)
# smiles_d['smiles'].loc[zinc_id].iloc[smiles_d.loc[zinc_id]['smiles_len'].argmin()]

## 读入小分子的结构

In [ ]:
ligands_file_name = "zinc/zinc_drug_like_3d_100k_rand.pdbqt.gz"
elements_list = ["C", "H", "O", "N", "S", "P", "BR", "CL", "F", "I"]
transform = partial(ele_filter, elements_list=elements_list)
ligands = ZincPdbqt(ligands_file_name, filter_=[transform])

In [ ]:
len(ligands.zinc_id)
with open("zinc_id.txt", "w") as f:
    f.write("\n".join(ligands.zinc_id))

## 构建数据集

In [ ]:
# ligands.zinc_id
# print(len(ligands))
# print(ligands.zinc_id[0])
# score_dict[ligands.zinc_id[500]]
# elements_list = ['C', 'H', 'O', 'N', 'S', 'P', 'BR', 'CL', 'F', 'I']
f = open("4z4d_100K_rand.txt", "w")
f.write("e(kcalmol^-1)\n")
data_dict = dict()
for ligands_id, ligands_model in tqdm(ligands, desc="prepare"):
    if ligands_id not in list(score_dict.keys()):
        print(f"here wrong {ligands_id}")
        continue
    data_dict["score"] = score_dict[ligands_id]
    data_dict["pos"] = generate_coor(ligands_model)
    data_dict["id"] = ligands_id
    f = write_data(f, data_dict)
    # break
f.close()

In [ ]:
score_dict = dict()
for i in range(102):
    ligands_file_name = f"/mnt/e/Python_Project/SMTarRNA/project/data/3a6p/3a6p_dock_origin_data/zinc_drug_like_3d_100k_dock_results_{i}.pdbqt.gz"
    ligands = ZincPdbqt(ligands_file_name)
    score_dict.update(ligands.scores)
    # break

In [ ]:
len(score_dict.keys())